# Bibliothèques

In [ ]:
import pickle
import cv2
from sklearn.neighbors import KNeighborsClassifier

# Chargement des données

In [ ]:
with open('data/visages.pkl', 'rb') as fh:
    visages = pickle.load(fh)
    visages = visages.reshape(len(visages), -1)

with open('data/prenoms.pkl', 'rb') as fh:
    noms = pickle.load(fh)
    
with open('data/admis.pkl', 'rb') as fh:
    admis = pickle.load(fh)
    
print('Shape of visages matrix --> ', visages.shape)
print(set(noms)) 

# Entraînement du modèle

In [ ]:
knn = KNeighborsClassifier(n_neighbors=4)
knn.fit(visages, noms)

# Détection des visages en temps réel

In [ ]:
def detection_visage(model, noms, admis):
    cascade_visage = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    camera = cv2.VideoCapture(0)
    
    while True:
        ret, frame = camera.read()
        
        if ret:
            coordonnees_visage = cascade_visage.detectMultiScale(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), 1.3, 5)
    
            for (x, y, l, h) in coordonnees_visage:
                visage = frame[y:y + h, x:x + l, :]
                visage_redimensionne = cv2.resize(visage, (50, 50)).flatten().reshape(1,-1)
                texte = model.predict(visage_redimensionne)
                if admis[noms.index(texte[0])]:
                    cv2.putText(frame, texte[0] + " (Admis)", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                    cv2.rectangle(frame, (x, y), (x + l, y + l), (0, 255, 0), 2)
                else:
                    cv2.putText(frame, texte[0] + " (Non Admis)", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                    cv2.rectangle(frame, (x, y), (x + l, y + l), (0, 0, 255), 2)
                
    
            cv2.imshow('Reconnaissance faciale en temps réel', frame)
            
            if cv2.waitKey(1) == 27:
                break
                
        else:
            
            print("erreur")
            break
    
    cv2.destroyAllWindows()
    camera.release()

In [ ]:
detection_visage(knn, noms, admis)